In [71]:
import os
import re
import subprocess
from pathlib import Path
import numpy as np
import command
import plotly.graph_objects as go

In [72]:
p = Path('logs')
directories = [x.name for x in p.iterdir() if x.is_dir()]
lines_dict = {}

In [73]:
def tail(f, n, offset=0):
    proc = subprocess.Popen(['tail', '-n', str(n + offset), f], stdout=subprocess.PIPE)
    lines = proc.stdout.readlines()
    return lines[:offset]

def return_result(f):
    result = [x.decode("utf-8") for x in tail(f, 7, 2)]
    result = [re.findall('\d+', result[0])[0]] + re.findall('\d+', result[1])
    result = [float(x) for x in result] # length of 7, [deg, wt pz/1-pz, 0.1, 0.3, 0.5, 0.7, 1.0]
    return result

In [74]:
for dir in directories:
    [N,K,l,s,n,exact] = [''.join(x for x in r if x.isdigit()) for r in dir.split('_')[:-1]]
    con = dir.split('_')[-1]
    exp_name = 'N='+N+',K='+K+',l='+l+',s='+s+',n='+n+',exact='+exact+','+con
    print(exp_name)
    q = p / dir
    files = list(q.glob('*.log'))
    pzs = [''.join(x for x in f.name if x.isdigit()) for f in files]
    pzs = np.array([float(pz) for pz in pzs]) / 100.0
    results = [return_result(f) for f in files]
    deg_err = np.array([x[0] for x in results]) / float(n)
    true_weight_err = np.array([x[1] for x in results]) / float(n)
    # sort so that the line will be smooth
    order = np.argsort(pzs)
    pzs = pzs[order]
    deg_err = deg_err[order]
    true_weight_err = true_weight_err[order]
    lines_dict[exp_name+',deg'] = (pzs, deg_err)
    lines_dict[exp_name+',wt'] = (pzs, true_weight_err)

N=128,K=65,l=16,s=23,n=10000,exact=0,PW


In [75]:
fig = go.Figure()

for x,y in lines_dict.items():
    fig.add_trace(go.Scatter(x=y[0], y=y[1], mode='lines+markers', name=x))

fig.show()